# 1. Imports

In [1]:
import copy
import numba
import matplotlib.pyplot as plt
plt.style.use('classic')
import numpy as np
import pandas as pd
import seaborn as sns 
import statistics
%matplotlib inline 
from os import getcwd
import plotly.express as px
import matplotlib.cm as cm
import pylab as pl

from scipy import sparse
from scipy import stats
from scipy import spatial

import sklearn
from sklearn import preprocessing
from sklearn import linear_model
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.metrics.pairwise import cosine_similarity

from surprise import accuracy
from surprise import BaselineOnly
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate
from surprise.model_selection import GridSearchCV
from surprise.prediction_algorithms.knns import KNNBasic

getcwd()

'C:\\Users\\tessa\\UVT Thesis\\Untitled Folder'

# 2. Preprocessing

In [2]:
data_inner = pd.read_csv('data_inner_preprocessed.csv')
testdata = pd.read_csv('testdata_inner.csv')

# 3. PCA

In [3]:
X = data_inner.drop('title', axis=1)
X = X.drop('artist', axis=1)
X = X.drop('song_id', axis=1)
X = X.drop('user_id', axis=1)
X = X.drop('album', axis=1)
X = X.drop('uri', axis=1)
X = X.drop('artist_name', axis=1)
X = X.drop('rel_rating', axis=1) # or play count
X = X.drop('year', axis=1)

testX = testdata.drop('title', axis=1)
testX = testX.drop('artist', axis=1)
testX = testX.drop('song_id', axis=1)
testX = testX.drop('user_id', axis=1)
testX = testX.drop('album', axis=1)
testX = testX.drop('uri', axis=1)
testX = testX.drop('artist_name', axis=1)
testX = testX.drop('rel_rating', axis=1) # or play count
testX = testX.drop('year', axis=1)

scalar = StandardScaler()
df_scaled = pd.DataFrame(scalar.fit_transform(X), columns=X.columns)
testdf_scaled = pd.DataFrame(scalar.fit_transform(testX), columns=testX.columns)

pca = PCA(n_components=8)
df_pca = pd.DataFrame(pca.fit_transform(df_scaled))

testpca = PCA(n_components=8)
testdf_pca = pd.DataFrame(testpca.fit_transform(testdf_scaled))# add user_id, song_id and play_count_categories

In [4]:
# add user_id, song_id and play_count

df_pca['user_id'] = data_inner['user_id']
df_pca['song_id'] = data_inner['song_id']
df_pca['rel_rating'] = data_inner['rel_rating']

testdf_pca['user_id'] = testdata['user_id']
testdf_pca['song_id'] = testdata['song_id']
testdf_pca['rel_rating'] = testdata['rel_rating']

In [5]:
df_pca_for_clustering = df_pca.drop('user_id', axis=1)
df_pca_for_clustering = df_pca_for_clustering.drop('song_id', axis=1)
df_pca_for_clustering = df_pca_for_clustering.drop('rel_rating', axis=1)

testdf_pca_for_clustering = testdf_pca.drop('user_id', axis=1)
testdf_pca_for_clustering = testdf_pca_for_clustering.drop('song_id', axis=1)
testdf_pca_for_clustering = testdf_pca_for_clustering.drop('rel_rating', axis=1)

# 3. Clustering

In [6]:
# then, we make the clusters
km = KMeans(n_clusters=7, init='k-means++', random_state=42)
song_cluster_pipeline = Pipeline([('scaler', StandardScaler()), 
                                  ('kmeans', km)],verbose=True)

song_cluster_pipeline.fit(df_pca_for_clustering)
song_cluster_labels = song_cluster_pipeline.predict(df_pca_for_clustering)
df_pca['cluster'] = song_cluster_labels

df_pca['cluster'].value_counts()

cluster0 = df_pca.loc[df_pca['cluster'] == 0]
print(cluster0.shape)

cluster1 = df_pca.loc[df_pca['cluster'] == 1]
print(cluster1.shape)

cluster2 = df_pca.loc[df_pca['cluster'] == 2]
print(cluster2.shape)

cluster3 = df_pca.loc[df_pca['cluster'] == 3]
print(cluster3.shape)

cluster4 = df_pca.loc[df_pca['cluster'] == 4]
print(cluster4.shape)

cluster5 = df_pca.loc[df_pca['cluster'] == 5]
print(cluster4.shape)

cluster6 = df_pca.loc[df_pca['cluster'] == 6]
print(cluster6.shape)

[Pipeline] ............ (step 1 of 2) Processing scaler, total=   0.2s
[Pipeline] ............ (step 2 of 2) Processing kmeans, total=  29.9s
(92058, 12)
(43882, 12)
(183790, 12)
(78633, 12)
(42769, 12)
(42769, 12)
(24330, 12)


In [7]:
test_song_cluster_labels = song_cluster_pipeline.predict(testdf_pca_for_clustering)

# assign cluster to testdata
testdf_pca['cluster'] = test_song_cluster_labels

testdf_pca['cluster'].value_counts()

# test clusters
testcluster0 = testdf_pca.loc[testdf_pca['cluster'] == 0]
print(testcluster0.shape)

testcluster1 = testdf_pca.loc[testdf_pca['cluster'] == 1]
print(testcluster1.shape)

testcluster2 = testdf_pca.loc[testdf_pca['cluster'] == 2]
print(testcluster2.shape)

testcluster3 = testdf_pca.loc[testdf_pca['cluster'] == 3]
print(testcluster3.shape)

testcluster4 = testdf_pca.loc[testdf_pca['cluster'] == 4]
print(testcluster4.shape)

testcluster5 = testdf_pca.loc[testdf_pca['cluster'] == 5]
print(testcluster5.shape)

testcluster6 = testdf_pca.loc[testdf_pca['cluster'] == 6]
print(testcluster6.shape)

(22707, 12)
(10956, 12)
(45965, 12)
(19545, 12)
(10564, 12)
(47307, 12)
(6208, 12)


In [8]:
dataset_for_clustering = df_pca.drop('user_id', axis=1)
dataset_for_clustering = dataset_for_clustering.drop('song_id', axis=1)

testdata_for_clustering = testdf_pca.drop('user_id', axis=1)
testdata_for_clustering = testdata_for_clustering.drop('song_id', axis=1)

# Cluster analysis (nog aanpassen na hyperparameter tuning K in K means)

In [9]:
# Calinski Harabasz score
from sklearn import metrics
from sklearn.metrics import pairwise_distances

labels_train = df_pca['cluster']
X_train = dataset_for_clustering
labels_test = testdf_pca['cluster']
X_test = testdata_for_clustering

print(metrics.calinski_harabasz_score(X_train, labels_train))
print(metrics.calinski_harabasz_score(X_test, labels_test))

# train: 95140.73783421429
# test: 23792.203526818364

107355.87607032365
26859.267605068886


In [10]:
# Davies Bouldin score
from sklearn.metrics import davies_bouldin_score

print(davies_bouldin_score(X_train, labels_train))
print(davies_bouldin_score(X_test, labels_test))

# train: 1.6584698292978697
# test: 1.6638013915958003

1.5009298071816903
1.5028252245400469


# 4. Weights

In [11]:
weight_c0 = cluster0.shape[0] / df_pca.shape[0]
weight_c1 = cluster1.shape[0] / df_pca.shape[0]
weight_c2 = cluster2.shape[0] / df_pca.shape[0]
weight_c3 = cluster3.shape[0] / df_pca.shape[0]
weight_c4 = cluster4.shape[0] / df_pca.shape[0]
weight_c5 = cluster5.shape[0] / df_pca.shape[0]
weight_c6 = cluster6.shape[0] / df_pca.shape[0]

weight_tc0 = testcluster0.shape[0] / testdata.shape[0]
weight_tc1 = testcluster1.shape[0] / testdata.shape[0]
weight_tc2 = testcluster2.shape[0] / testdata.shape[0]
weight_tc3 = testcluster3.shape[0] / testdata.shape[0]
weight_tc4 = testcluster4.shape[0] / testdata.shape[0]
weight_tc5 = testcluster5.shape[0] / testdata.shape[0]
weight_tc6 = testcluster6.shape[0] / testdata.shape[0]

# 5. KNN

In [12]:
sim_options = {'name': 'cosine',
              'user_based': False}

model_knn = KNNBasic(sim_options=sim_options, k=14, verbose=True)

In [13]:
reader = Reader(line_format = 'user item rating', rating_scale=(0, 35))
cluster0 = Dataset.load_from_df(cluster0[['user_id', 'song_id', 'rel_rating']], reader = reader) # deze toe gaan voegen
cluster1 = Dataset.load_from_df(cluster1[['user_id', 'song_id', 'rel_rating']], reader = reader)
cluster2 = Dataset.load_from_df(cluster2[['user_id', 'song_id', 'rel_rating']], reader = reader)
cluster3 = Dataset.load_from_df(cluster3[['user_id', 'song_id', 'rel_rating']], reader = reader)
cluster4 = Dataset.load_from_df(cluster4[['user_id', 'song_id', 'rel_rating']], reader = reader)
cluster5 = Dataset.load_from_df(cluster5[['user_id', 'song_id', 'rel_rating']], reader = reader)
cluster6 = Dataset.load_from_df(cluster6[['user_id', 'song_id', 'rel_rating']], reader = reader)

type(cluster0)

# split data into train and validation data. so now we have train, validation and test data.
from surprise.model_selection import train_test_split

train_cluster0, val_cluster0 = train_test_split(cluster0, test_size=0.25, random_state=42)
train_cluster1, val_cluster1 = train_test_split(cluster1, test_size=0.25, random_state=42)
train_cluster2, val_cluster2 = train_test_split(cluster2, test_size=0.25, random_state=42)
train_cluster3, val_cluster3 = train_test_split(cluster3, test_size=0.25, random_state=42)
train_cluster4, val_cluster4 = train_test_split(cluster4, test_size=0.25, random_state=42)
train_cluster5, val_cluster5 = train_test_split(cluster5, test_size=0.25, random_state=42)
train_cluster6, val_cluster6 = train_test_split(cluster6, test_size=0.25, random_state=42)

# get train set in a different format
exc00, newtrain_cluster0 = train_test_split(cluster0, test_size=0.999, random_state=42)
exc11, newtrain_cluster1 = train_test_split(cluster1, test_size=0.999, random_state=42)
exc22, newtrain_cluster2 = train_test_split(cluster2, test_size=0.999, random_state=42)
exc33, newtrain_cluster3 = train_test_split(cluster3, test_size=0.999, random_state=42)
exc44, newtrain_cluster4 = train_test_split(cluster4, test_size=0.999, random_state=42)
exc55, newtrain_cluster5 = train_test_split(cluster5, test_size=0.999, random_state=42)
exc66, newtrain_cluster6 = train_test_split(cluster6, test_size=0.999, random_state=42)
print(type(exc00), type(train_cluster0))

print(type(train_cluster0), type(val_cluster0))

reader = Reader(line_format = 'user item rating', rating_scale=(0, 35))
test_cluster0 = Dataset.load_from_df(testcluster0[['user_id', 'song_id', 'rel_rating']], reader = reader)
test_cluster1 = Dataset.load_from_df(testcluster1[['user_id', 'song_id', 'rel_rating']], reader = reader)
test_cluster2 = Dataset.load_from_df(testcluster2[['user_id', 'song_id', 'rel_rating']], reader = reader)
test_cluster3 = Dataset.load_from_df(testcluster3[['user_id', 'song_id', 'rel_rating']], reader = reader)
test_cluster4 = Dataset.load_from_df(testcluster4[['user_id', 'song_id', 'rel_rating']], reader = reader)
test_cluster5 = Dataset.load_from_df(testcluster5[['user_id', 'song_id', 'rel_rating']], reader = reader)
test_cluster6 = Dataset.load_from_df(testcluster6[['user_id', 'song_id', 'rel_rating']], reader = reader)

type(test_cluster0)

# split data into test and excess dataset to get a different type
from surprise.model_selection import train_test_split

exc0, test_cluster0 = train_test_split(test_cluster0, test_size=0.999, random_state=42)
exc1, test_cluster1 = train_test_split(test_cluster1, test_size=0.999, random_state=42)
exc2, test_cluster2 = train_test_split(test_cluster2, test_size=0.999, random_state=42)
exc3, test_cluster3 = train_test_split(test_cluster3, test_size=0.999, random_state=42)
exc4, test_cluster4 = train_test_split(test_cluster4, test_size=0.999, random_state=42)
exc5, test_cluster5 = train_test_split(test_cluster5, test_size=0.999, random_state=42)
exc6, test_cluster6 = train_test_split(test_cluster6, test_size=0.999, random_state=42)

print(type(exc0), type(test_cluster0))

<class 'surprise.trainset.Trainset'> <class 'surprise.trainset.Trainset'>
<class 'surprise.trainset.Trainset'> <class 'list'>
<class 'surprise.trainset.Trainset'> <class 'list'>


In [14]:
# find the train mae
prediction0_0 = model_knn.fit(train_cluster0).test(newtrain_cluster0)
prediction1_0 = model_knn.fit(train_cluster1).test(newtrain_cluster1)
prediction2_0 = model_knn.fit(train_cluster2).test(newtrain_cluster2)
prediction3_0 = model_knn.fit(train_cluster3).test(newtrain_cluster3)
prediction4_0 = model_knn.fit(train_cluster4).test(newtrain_cluster4)
prediction5_0 = model_knn.fit(train_cluster5).test(newtrain_cluster5)
prediction6_0 = model_knn.fit(train_cluster6).test(newtrain_cluster6)

# find the validation set mae
prediction0_1 = model_knn.fit(train_cluster0).test(val_cluster0)
prediction1_1 = model_knn.fit(train_cluster1).test(val_cluster1)
prediction2_1 = model_knn.fit(train_cluster2).test(val_cluster2)
prediction3_1 = model_knn.fit(train_cluster3).test(val_cluster3)
prediction4_1 = model_knn.fit(train_cluster4).test(val_cluster4)
prediction5_1 = model_knn.fit(train_cluster5).test(val_cluster5)
prediction6_1 = model_knn.fit(train_cluster6).test(val_cluster6)

# check mae for test set for model comparison
prediction0_2 = model_knn.fit(train_cluster0).test(test_cluster0)
prediction1_2 = model_knn.fit(train_cluster1).test(test_cluster1)
prediction2_2 = model_knn.fit(train_cluster2).test(test_cluster2)
prediction3_2 = model_knn.fit(train_cluster3).test(test_cluster3)
prediction4_2 = model_knn.fit(train_cluster4).test(test_cluster4)
prediction5_2 = model_knn.fit(train_cluster5).test(test_cluster5)
prediction6_2 = model_knn.fit(train_cluster6).test(test_cluster6)

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing th

In [15]:
## average prediction frame

In [16]:
# VALIDATION DATA
result0 = pd.DataFrame(prediction0_1, columns=['visitor_id', 'item_id', 'base_event', 'predict_event', 'details'])
result0.drop(columns = {'details'}, inplace=True)
result0['error'] = abs(result0['base_event'] - result0['predict_event'])
result0.head()

result1 = pd.DataFrame(prediction1_1, columns=['visitor_id', 'item_id', 'base_event', 'predict_event', 'details'])
result1.drop(columns = {'details'}, inplace=True)
result1['error'] = abs(result1['base_event'] - result1['predict_event'])
result1.head()

result2 = pd.DataFrame(prediction2_1, columns=['visitor_id', 'item_id', 'base_event', 'predict_event', 'details'])
result2.drop(columns = {'details'}, inplace=True)
result2['error'] = abs(result2['base_event'] - result2['predict_event'])
result2.head()

result3 = pd.DataFrame(prediction3_1, columns=['visitor_id', 'item_id', 'base_event', 'predict_event', 'details'])
result3.drop(columns = {'details'}, inplace=True)
result3['error'] = abs(result3['base_event'] - result3['predict_event'])
result3.head()

result4 = pd.DataFrame(prediction4_1, columns=['visitor_id', 'item_id', 'base_event', 'predict_event', 'details'])
result4.drop(columns = {'details'}, inplace=True)
result4['error'] = abs(result4['base_event'] - result4['predict_event'])
result4.head()

result5 = pd.DataFrame(prediction5_1, columns=['visitor_id', 'item_id', 'base_event', 'predict_event', 'details'])
result5.drop(columns = {'details'}, inplace=True)
result5['error'] = abs(result5['base_event'] - result5['predict_event'])
result5.head()

result6 = pd.DataFrame(prediction6_1, columns=['visitor_id', 'item_id', 'base_event', 'predict_event', 'details'])
result6.drop(columns = {'details'}, inplace=True)
result6['error'] = abs(result6['base_event'] - result6['predict_event'])
result6.head()

,visitor_id,item_id,base_event,predict_event,error
0,6633572523ef305dce0eef5e5130118227847dfc,SOLFXKT12AB017E3E0,0.497409,0.365531,0.131878
1,0c7b202b1d20194deedf7d0d086792ce4feff730,SOKMHKY12AF72AB079,2.255962,1.780171,0.475791
2,8386bd8058028e5787d6956d29f3a54a4965d8ef,SOBONKR12A58A7A7E0,2.923567,0.343439,2.580128
3,185beb26cdfd58f67e288554baf43fbae074f31f,SOXMGFH12A6701FF31,1.924342,2.180921,0.256579
4,028cb6450c0dd25109384c4240fce5987b85ee58,SOGPBAW12A6D4F9F22,0.738636,2.444726,1.706090


In [33]:
# TRAIN DATA
result0 = pd.DataFrame(prediction0_0, columns=['visitor_id', 'item_id', 'base_event', 'predict_event', 'details'])
result0.drop(columns = {'details'}, inplace=True)
result0['error'] = abs(result0['base_event'] - result0['predict_event'])
result0.head()

result1 = pd.DataFrame(prediction1_0, columns=['visitor_id', 'item_id', 'base_event', 'predict_event', 'details'])
result1.drop(columns = {'details'}, inplace=True)
result1['error'] = abs(result1['base_event'] - result1['predict_event'])
result1.head()

result2 = pd.DataFrame(prediction2_0, columns=['visitor_id', 'item_id', 'base_event', 'predict_event', 'details'])
result2.drop(columns = {'details'}, inplace=True)
result2['error'] = abs(result2['base_event'] - result2['predict_event'])
result2.head()

result3 = pd.DataFrame(prediction3_0, columns=['visitor_id', 'item_id', 'base_event', 'predict_event', 'details'])
result3.drop(columns = {'details'}, inplace=True)
result3['error'] = abs(result3['base_event'] - result3['predict_event'])
result3.head()

result4 = pd.DataFrame(prediction4_0, columns=['visitor_id', 'item_id', 'base_event', 'predict_event', 'details'])
result4.drop(columns = {'details'}, inplace=True)
result4['error'] = abs(result4['base_event'] - result4['predict_event'])
result4.head()

result5 = pd.DataFrame(prediction5_0, columns=['visitor_id', 'item_id', 'base_event', 'predict_event', 'details'])
result5.drop(columns = {'details'}, inplace=True)
result5['error'] = abs(result5['base_event'] - result5['predict_event'])
result5.head()

result6 = pd.DataFrame(prediction6_0, columns=['visitor_id', 'item_id', 'base_event', 'predict_event', 'details'])
result6.drop(columns = {'details'}, inplace=True)
result6['error'] = abs(result6['base_event'] - result6['predict_event'])
result6.head()

,visitor_id,item_id,base_event,predict_event,error
0,f193dd1a1539a00a119aad26b08c586df3637d4e,SOXTGXZ12A6D4F9795,0.996633,1.578002,0.581369
1,59852abaf8a5cd2e14a5d3eed918dbf6751b47eb,SODBDBV12A6310E14A,0.779221,0.740260,0.038961
2,5bd155e41faf2d24c95b45e5bd558cebf7fe892c,SOPUJGO12AB017CE27,3.818182,3.818182,0.000000
3,46d8ac0df74a628a674ce337e862911755ba444b,SOIFBGG12A8C136157,0.051282,0.051282,0.000000
4,9f2ab1340d6063152daeab9a36882b5302e1e711,SOTDYAT12A8C13A26A,1.559538,0.668373,0.891164


In [44]:
# TEST DATA
result0 = pd.DataFrame(prediction0_2, columns=['visitor_id', 'item_id', 'base_event', 'predict_event', 'details'])
result0.drop(columns = {'details'}, inplace=True)
result0['error'] = abs(result0['base_event'] - result0['predict_event'])
result0.head()

result1 = pd.DataFrame(prediction1_2, columns=['visitor_id', 'item_id', 'base_event', 'predict_event', 'details'])
result1.drop(columns = {'details'}, inplace=True)
result1['error'] = abs(result1['base_event'] - result1['predict_event'])
result1.head()

result2 = pd.DataFrame(prediction2_2, columns=['visitor_id', 'item_id', 'base_event', 'predict_event', 'details'])
result2.drop(columns = {'details'}, inplace=True)
result2['error'] = abs(result2['base_event'] - result2['predict_event'])
result2.head()

result3 = pd.DataFrame(prediction3_2, columns=['visitor_id', 'item_id', 'base_event', 'predict_event', 'details'])
result3.drop(columns = {'details'}, inplace=True)
result3['error'] = abs(result3['base_event'] - result3['predict_event'])
result3.head()

result4 = pd.DataFrame(prediction4_2, columns=['visitor_id', 'item_id', 'base_event', 'predict_event', 'details'])
result4.drop(columns = {'details'}, inplace=True)
result4['error'] = abs(result4['base_event'] - result4['predict_event'])
result4.head()

result5 = pd.DataFrame(prediction5_2, columns=['visitor_id', 'item_id', 'base_event', 'predict_event', 'details'])
result5.drop(columns = {'details'}, inplace=True)
result5['error'] = abs(result5['base_event'] - result5['predict_event'])
result5.head()

result6 = pd.DataFrame(prediction6_2, columns=['visitor_id', 'item_id', 'base_event', 'predict_event', 'details'])
result6.drop(columns = {'details'}, inplace=True)
result6['error'] = abs(result6['base_event'] - result6['predict_event'])
result6.head()

,visitor_id,item_id,base_event,predict_event,error
0,b161e5a439710f3fa1da4cb624bfeedf7769019d,SOAMFCO12A6701F117,1.097015,2.444726,1.347711
1,eac355b50aca9824ea8b698833711ee517d8d954,SOGTQNI12AB0184A5C,1.647059,1.294118,0.352941
2,be258b7a65abfe918c874e282e91dbd7837e7e6b,SOOLULU12A6701FBC4,0.574850,0.932200,0.357350
3,37c3f9e93125c347b34d1b73ac704af790a122f4,SOGBKFC12AB01843FC,2.758621,2.758621,0.000000
4,d940c02688b9518c0060ad0548a80ef33263f39c,SOBDZKO12A67020664,0.362573,0.401420,0.038847


In [45]:
## datasets anders mergen
play_count_categories = []

for number in result0['base_event']:
    if number < 1:
        play_count_categories.append(0)
    else:
        play_count_categories.append(1)
        
result0 = result0.assign(base_binary = play_count_categories)

play_count_categories1 = []

for number in result0['predict_event']:
    if number < 1:
        play_count_categories1.append(0)
    else:
        play_count_categories1.append(1)
        
result0 = result0.assign(predict_binary = play_count_categories1)

result0.head()

,visitor_id,item_id,base_event,predict_event,error,base_binary,predict_binary
0,3e85d31d1a665a32357947850d810d41f576e138,SOOYQXS12A6D4F84B9,1.428571,1.428571,0.000000,1,1
1,18fd5364b431023cd1d6e469dc11140688106a21,SOWWCNJ12A81C1FFA5,0.750000,1.070800,0.320800,0,1
2,fd1f275f7c4f69a015ffa07039df35e6621020b0,SONDBUE12A58A75F7A,0.762195,0.762195,0.000000,0,0
3,9001590c1387c735f132ed087c24c191d57520ff,SOCLMAD12AB017FC09,1.270492,1.058543,0.211949,1,1
4,00e7e7ae22cde9cd5e8c98e2b159270747613c40,SOKHURT12AB0185DE0,0.312000,0.437615,0.125615,0,0


In [46]:
play_count_categories = []

for number in result1['base_event']:
    if number < 1:
        play_count_categories.append(0)
    else:
        play_count_categories.append(1)
        
result1 = result1.assign(base_binary = play_count_categories)

play_count_categories1 = []

for number in result1['predict_event']:
    if number < 1:
        play_count_categories1.append(0)
    else:
        play_count_categories1.append(1)
        
result1 = result1.assign(predict_binary = play_count_categories1)

result1.head()

,visitor_id,item_id,base_event,predict_event,error,base_binary,predict_binary
0,ab32eff76c81a39a09c1f10dd2155e7b3372a419,SOOVANF12A6D4FA500,0.210826,1.054131,0.843305,0,1
1,a1a4acbddf1b309b037b17be31370c1997f1c906,SOCVTLJ12A6310F0FD,4.012739,0.742244,3.270495,1,0
2,4fa22edbfd4fcebb74956c8701e20b15fa1fc3e8,SOQGOPT12AAF3B2B27,0.636248,0.681695,0.045446,0,0
3,249ae19e39f6a2caa3d17ce90e028d04555a67aa,SODGJKH12AAA8C9487,0.521212,0.557648,0.036436,0,0
4,fa91a437c5e2bc948651e0cc25708c8f5a8c3c18,SOCYDJG12A8C13E343,4.599823,0.255546,4.344277,1,0


In [47]:
play_count_categories = []

for number in result2['base_event']:
    if number < 1:
        play_count_categories.append(0)
    else:
        play_count_categories.append(1)
        
result2 = result2.assign(base_binary = play_count_categories)

play_count_categories1 = []

for number in result2['predict_event']:
    if number < 1:
        play_count_categories1.append(0)
    else:
        play_count_categories1.append(1)
        
result2 = result2.assign(predict_binary = play_count_categories1)
result2.head()

,visitor_id,item_id,base_event,predict_event,error,base_binary,predict_binary
0,961fb1b642439b10b08b68b2909607bc3e716e3f,SOBXHDL12A81C204C0,0.260870,0.746170,0.485300,0,0
1,5db818d82beeffdb6ede78664b4012064784c232,SOBJBEU12AF729D526,4.208696,0.382609,3.826087,1,0
2,c44097fef4158d02a10b61a4fe2908bcb73cc306,SOHEMBB12A6701E907,0.103448,0.103448,0.000000,0,0
3,260c88f5bc6af9e1ac97d9ef323a610b30208254,SOYEZKY12AB018909A,1.000000,1.000000,0.000000,1,1
4,dc4d8836266282b6d7dbff8e19ff10b7dd71cb8f,SOJQCAO12AB0183B8A,0.777778,0.777778,0.000000,0,0


In [48]:
play_count_categories = []

for number in result3['base_event']:
    if number < 1:
        play_count_categories.append(0)
    else:
        play_count_categories.append(1)
        
result3 = result3.assign(base_binary = play_count_categories)

play_count_categories1 = []

for number in result3['predict_event']:
    if number < 1:
        play_count_categories1.append(0)
    else:
        play_count_categories1.append(1)
        
result3 = result3.assign(predict_binary = play_count_categories1)
result3.head()

,visitor_id,item_id,base_event,predict_event,error,base_binary,predict_binary
0,908158c14e8a4760c9541ec2aa0c3012445ddb67,SOUCBEB12A6310E1F9,0.686567,0.753782,0.067215,0,0
1,394e6fa424241c6b714201aefb22e678f0457423,SOYMCQU12A6D4F6FC8,0.476190,0.687847,0.211657,0,0
2,fef45cbcef5bf7571d053652c9c46085a3ee38a4,SOONJMQ12A8C137E6D,1.126437,0.835528,0.290909,1,0
3,e372bac4b61366648f76bdc4f2f47e6a66da818b,SONYKOW12AB01849C9,1.320388,1.320388,0.000000,1,1
4,c536035e4a3b680fd3549631d817b26b1e242ab1,SODGVGW12AC9075A8D,0.400000,0.633937,0.233937,0,0


In [49]:
play_count_categories = []

for number in result4['base_event']:
    if number < 1:
        play_count_categories.append(0)
    else:
        play_count_categories.append(1)
        
result4 = result4.assign(base_binary = play_count_categories)

play_count_categories1 = []

for number in result4['predict_event']:
    if number < 1:
        play_count_categories1.append(0)
    else:
        play_count_categories1.append(1)
        
result4 = result4.assign(predict_binary = play_count_categories1)
result4.head()

,visitor_id,item_id,base_event,predict_event,error,base_binary,predict_binary
0,4c5e7d2974cbe5ee8a4f7ee484523f85aeccb44a,SOHHALP12A6D4FB6A8,0.510204,0.510204,0.000000,0,0
1,a2f7a033ef843b952d7a0c45aa8ae03d3ee25c83,SORELVF12AF72A22DF,0.260450,0.390675,0.130225,0,0
2,17b01d193ddb8c89de54033919a410cfab33f0f1,SOUJVIT12A8C1451C1,2.575000,2.298862,0.276138,1,1
3,7909446465dc109053091f80614a2001c505d6b2,SOVYNZA12A8C13C378,0.753943,0.973312,0.219369,0,0
4,ce437bd511ae1fb47cac33c55818395657eaa1cc,SORYLNP12A8C13C3E7,0.120253,0.120253,0.000000,0,0


In [50]:
# do not merge

## result5
play_count_categories = []

for number in result5['base_event']:
    if number < 1:
        play_count_categories.append(0)
    else:
        play_count_categories.append(1)
        
result5 = result5.assign(base_binary = play_count_categories)

play_count_categories1 = []

for number in result5['predict_event']:
    if number < 1:
        play_count_categories1.append(0)
    else:
        play_count_categories1.append(1)
        
result5 = result5.assign(predict_binary = play_count_categories1)

result5.head()

,visitor_id,item_id,base_event,predict_event,error,base_binary,predict_binary
0,62f00a3dbbc00b50f2aae5bd899bb0136d42f24f,SOWXVAW12A8C1414FE,6.433673,1.790417,4.643257,1,1
1,0feb689a479f0316e0e1715f323dea60ce63e3cb,SOUTGUB12AB017DC5B,0.592593,0.592593,0.000000,0,0
2,2132e9e50fa7cd9dedfd38a3109b5b83e87ae97b,SOOAOHX12A6D4F8E8C,0.818182,0.975768,0.157586,0,0
3,f16a7514487861ae5543fd6765a56b27fe512210,SOJYBJZ12AB01801D0,0.311111,0.311111,0.000000,0,0
4,a34decc9bbcfa98f1840fa874e2289d485999e77,SOZVCRW12A67ADA0B7,0.941176,0.941176,0.000000,0,0


In [51]:
# do not merge

## result6
play_count_categories = []

for number in result6['base_event']:
    if number < 1:
        play_count_categories.append(0)
    else:
        play_count_categories.append(1)
        
result6 = result6.assign(base_binary = play_count_categories)

play_count_categories1 = []

for number in result6['predict_event']:
    if number < 1:
        play_count_categories1.append(0)
    else:
        play_count_categories1.append(1)
        
result6 = result6.assign(predict_binary = play_count_categories1)

result6.head()

,visitor_id,item_id,base_event,predict_event,error,base_binary,predict_binary
0,b161e5a439710f3fa1da4cb624bfeedf7769019d,SOAMFCO12A6701F117,1.097015,2.444726,1.347711,1,1
1,eac355b50aca9824ea8b698833711ee517d8d954,SOGTQNI12AB0184A5C,1.647059,1.294118,0.352941,1,1
2,be258b7a65abfe918c874e282e91dbd7837e7e6b,SOOLULU12A6701FBC4,0.574850,0.932200,0.357350,0,0
3,37c3f9e93125c347b34d1b73ac704af790a122f4,SOGBKFC12AB01843FC,2.758621,2.758621,0.000000,1,1
4,d940c02688b9518c0060ad0548a80ef33263f39c,SOBDZKO12A67020664,0.362573,0.401420,0.038847,0,0


In [52]:
# calculate accuracy
from sklearn.metrics import confusion_matrix

y_actual0 = pd.Series(result0['base_binary'], name='actual')
y_predict0 = pd.Series(result0['predict_binary'], name='predicted')
con_mat0 = (pd.crosstab(y_actual0, y_predict0)) / (pd.crosstab(y_actual0, y_predict0)).sum(axis=1)

y_actual1 = pd.Series(result1['base_binary'], name='actual')
y_predict1 = pd.Series(result1['predict_binary'], name='predicted')
con_mat1 = (pd.crosstab(y_actual1, y_predict1)) / (pd.crosstab(y_actual1, y_predict1)).sum(axis=1)

y_actual2 = pd.Series(result2['base_binary'], name='actual')
y_predict2 = pd.Series(result2['predict_binary'], name='predicted')
con_mat2 = (pd.crosstab(y_actual2, y_predict2)) / (pd.crosstab(y_actual2, y_predict2)).sum(axis=1)

y_actual3 = pd.Series(result3['base_binary'], name='actual')
y_predict3 = pd.Series(result3['predict_binary'], name='predicted')
con_mat3 = (pd.crosstab(y_actual3, y_predict3)) / (pd.crosstab(y_actual3, y_predict3)).sum(axis=1)

y_actual4 = pd.Series(result4['base_binary'], name='actual')
y_predict4 = pd.Series(result4['predict_binary'], name='predicted')
con_mat4 = (pd.crosstab(y_actual4, y_predict4)) / (pd.crosstab(y_actual4, y_predict4)).sum(axis=1)

y_actual5 = pd.Series(result5['base_binary'], name='actual')
y_predict5 = pd.Series(result5['predict_binary'], name='predicted')
con_mat5 = (pd.crosstab(y_actual5, y_predict5)) / (pd.crosstab(y_actual5, y_predict5)).sum(axis=1)

y_actual6 = pd.Series(result6['base_binary'], name='actual')
y_predict6 = pd.Series(result6['predict_binary'], name='predicted')
con_mat6 = (pd.crosstab(y_actual6, y_predict6)) / (pd.crosstab(y_actual6, y_predict6)).sum(axis=1)

In [53]:
print('Result 0:', con_mat0)
print('\n')

print('Result 1:', con_mat1)
print('\n')

print('Result 2:', con_mat2)
print('\n')

print('Result 3:', con_mat3)
print('\n')

print('Result 4:', con_mat4)
print('\n')

print('Result 5:', con_mat5)
print('\n')

print('Result 6:', con_mat6)
print('\n')

Result 0: predicted         0         1
actual                       
0          0.840712  0.386055
1          0.144094  0.650770


Result 1: predicted         0         1
actual                       
0          0.781269  0.626987
1          0.210228  0.397386


Result 2: predicted         0         1
actual                       
0          0.813771  0.468167
1          0.149306  0.624656


Result 3: predicted         0         1
actual                       
0          0.875267  0.318413
1          0.127085  0.675582


Result 4: predicted         0         1
actual                       
0          0.893176  0.249053
1          0.134850  0.685606


Result 5: predicted         0         1
actual                       
0          0.813700  0.463434
1          0.148951  0.629474


Result 6: predicted         0         1
actual                       
0          0.538501  0.213393
1          0.162672  0.924782




In [54]:
TP = (con_mat0[1][1])*weight_tc0 + (con_mat1[1][1])*weight_tc1 + (con_mat2[1][1])*weight_tc2 + (con_mat3[1][1])*weight_tc3 + (con_mat4[1][1])*weight_tc4 + (con_mat5[1][1])*weight_tc5 + (con_mat6[1][1])*weight_tc6 
TN = (con_mat0[0][0])*weight_tc0 + (con_mat0[0][0])*weight_tc1 + (con_mat0[0][0])*weight_tc2 + (con_mat0[0][0])*weight_tc3 + (con_mat0[0][0])*weight_tc4 + (con_mat0[0][0])*weight_tc5 + (con_mat6[0][0])*weight_tc6 
FP = (con_mat0[0][1])*weight_tc0 + (con_mat1[0][1])*weight_tc1 + (con_mat2[0][1])*weight_tc2 + (con_mat3[0][1])*weight_tc3 + (con_mat4[0][1])*weight_tc4 + (con_mat5[0][1])*weight_tc5 + (con_mat6[0][1])*weight_tc6 
FN = (con_mat0[1][0])*weight_tc0 + (con_mat1[1][0])*weight_tc1 + (con_mat2[1][0])*weight_tc2 + (con_mat3[1][0])*weight_tc3 + (con_mat4[1][0])*weight_tc4 + (con_mat5[1][0])*weight_tc5 + (con_mat6[1][0])*weight_tc6 

print('TP:',TP,'\nTN:',TN,'\nFP:',FP,'\nFN:',FN)

TP: 0.635885921558148 
TN: 0.8292201423167838 
FP: 0.14947918074673233 
FN: 0.4242368537612949


In [55]:
from sklearn import metrics
recall0 = metrics.recall_score(y_actual0, y_predict0)
recall1 = metrics.recall_score(y_actual1, y_predict1)
recall2 = metrics.recall_score(y_actual2, y_predict2)
recall3 = metrics.recall_score(y_actual3, y_predict3)
recall4 = metrics.recall_score(y_actual4, y_predict4)
recall5 = metrics.recall_score(y_actual5, y_predict5)
recall6 = metrics.recall_score(y_actual6, y_predict6)
recall = recall0*weight_tc0 + recall1*weight_tc1 + recall2*weight_tc2 + recall3*weight_tc3 + recall4*weight_tc4 + recall5*weight_tc5 + recall6*weight_tc6

precision0 = metrics.precision_score(y_actual0, y_predict0)
precision1 = metrics.precision_score(y_actual1, y_predict1)
precision2 = metrics.precision_score(y_actual2, y_predict2)
precision3 = metrics.precision_score(y_actual3, y_predict3)
precision4 = metrics.precision_score(y_actual4, y_predict4)
precision5 = metrics.precision_score(y_actual5, y_predict5)
precision6 = metrics.precision_score(y_actual6, y_predict6)
precision = precision0*weight_tc0 + precision1*weight_tc1 + precision2*weight_tc2 + precision3*weight_tc3 + precision4*weight_tc4 + precision5*weight_tc5 + precision6*weight_tc6

accuracy0 = metrics.accuracy_score(y_actual0, y_predict0)
accuracy1 = metrics.accuracy_score(y_actual1, y_predict1)
accuracy2 = metrics.accuracy_score(y_actual2, y_predict2)
accuracy3 = metrics.accuracy_score(y_actual3, y_predict3)
accuracy4 = metrics.accuracy_score(y_actual4, y_predict4)
accuracy5 = metrics.accuracy_score(y_actual5, y_predict5)
accuracy6 = metrics.accuracy_score(y_actual6, y_predict6)
accuracy = accuracy0*weight_tc0 + accuracy1*weight_tc1 + accuracy2*weight_tc2 + accuracy3*weight_tc3 + accuracy4*weight_tc4 + accuracy5*weight_tc5 + accuracy6*weight_tc6

fscore0 = metrics.f1_score(y_actual0, y_predict0)
fscore1 = metrics.f1_score(y_actual1, y_predict1)
fscore2 = metrics.f1_score(y_actual2, y_predict2)
fscore3 = metrics.f1_score(y_actual3, y_predict3)
fscore4 = metrics.f1_score(y_actual4, y_predict4)
fscore5 = metrics.f1_score(y_actual5, y_predict5)
fscore6 = metrics.f1_score(y_actual6, y_predict6)
fscore = fscore0*weight_tc0 + fscore1*weight_tc1 + fscore2*weight_tc2 + fscore3*weight_tc3 + fscore4*weight_tc4 + fscore5*weight_tc5 + fscore6*weight_tc6

print('Recall:', round(recall,3), '\nPrecision:', round(precision,3), '\nAccuracy:', round(accuracy,3), '\nF1 score:', 
      round(fscore,3))

Recall: 0.636 
Precision: 0.601 
Accuracy: 0.772 
F1 score: 0.617


In [56]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# NOG VAN APARTE CLUSTERS MAKEN
conmat0 = confusion_matrix(y_actual0, y_predict0)
conmat1 = confusion_matrix(y_actual1, y_predict1)
conmat2 = confusion_matrix(y_actual2, y_predict2)
conmat3 = confusion_matrix(y_actual3, y_predict3)
conmat4 = confusion_matrix(y_actual4, y_predict4)
conmat5 = confusion_matrix(y_actual5, y_predict5)
conmat6 = confusion_matrix(y_actual6, y_predict6)

TP = (con_mat0[1][1])*weight_tc0 + (con_mat1[1][1])*weight_tc1 + (con_mat2[1][1])*weight_tc2 + (con_mat3[1][1])*weight_tc3 + (con_mat4[1][1])*weight_tc4 + (con_mat5[1][1])*weight_tc5 + (con_mat6[1][1])*weight_tc6 
TN = (con_mat0[0][0])*weight_tc0 + (con_mat1[0][0])*weight_tc1 + (con_mat2[0][0])*weight_tc2 + (con_mat3[0][0])*weight_tc3 + (con_mat4[0][0])*weight_tc4 + (con_mat1[0][0])*weight_tc5 + (con_mat6[0][0])*weight_tc6
FP = (con_mat0[0][1])*weight_tc0 + (con_mat1[0][1])*weight_tc1 + (con_mat2[0][1])*weight_tc2 + (con_mat3[0][1])*weight_tc3 + (con_mat4[0][1])*weight_tc4 + (con_mat5[0][1])*weight_tc5 + (con_mat6[0][1])*weight_tc6 
FN = (con_mat0[1][0])*weight_tc0 + (con_mat1[1][0])*weight_tc1 + (con_mat2[1][0])*weight_tc2 + (con_mat3[1][0])*weight_tc3 + (con_mat4[1][0])*weight_tc4 + (con_mat5[1][0])*weight_tc5 + (con_mat6[1][0])*weight_tc6 

print('TP', TP, '\nTN', TN, '\nFP', FP, '\nFN', FN)
TPshare = TP/(TP + TN + FP + FN)
TNshare = TN/(TP + TN + FP + FN)
FPshare = FP/(TP + TN + FP + FN)
FNshare = FN/(TP + TN + FP + FN)
print('TP', TPshare, '\nTN', TNshare, '\nFP', FPshare, '\nFN', FNshare)

TP 0.635885921558148 
TN 0.8079518146055054 
FP 0.14947918074673233 
FN 0.4242368537612949
TP 0.31517669110075314 
TN 0.4004611060931097 
FP 0.0740893169340255 
FN 0.21027288587211168


In [57]:
print(conmat0)
print(conmat1)
print(conmat2)
print(conmat3)
print(conmat4)
print(conmat5)
print(conmat6)

[[13501  2558]
 [ 2314  4312]]
[[6340 1775]
 [1706 1125]]
[[26734  6118]
 [ 4905  8163]]
[[12280  1750]
 [ 1783  3713]]
[[6597  789]
 [ 996 2172]]
[[27429  6280]
 [ 5021  8530]]
[[1056  905]
 [ 319 3922]]


In [58]:
ttl_TP = conmat0[1][1] + conmat1[1][1] + conmat2[1][1] + conmat3[1][1] + conmat5[1][1] + conmat4[1][1] + conmat6[1][1]  
ttl_TN = conmat0[0][0] + conmat1[0][0] + conmat2[0][0] + conmat3[0][0]+ conmat4[0][0] + conmat5[0][0] + conmat6[0][0]
ttl_FP = conmat0[0][1] + conmat1[0][1] + conmat2[0][1] + conmat3[0][1]+ conmat4[0][1] + conmat5[0][1] + conmat6[0][1]
ttl_FN = conmat0[1][0] + conmat1[1][0] + conmat2[1][0] + conmat3[1][0]+ conmat4[1][0] + conmat5[1][0] + conmat6[1][0]

print(ttl_TP, ttl_TN, ttl_FP, ttl_FN)

31937 93937 20175 17044


In [59]:
"""[[13501  2558]
 [ 2314  4312]]
[[6340 1775]
 [1706 1125]]
[[26734  6118]
 [ 4905  8163]]
[[12280  1750]
 [ 1783  3713]]
[[6597  789]
 [ 996 2172]]
[[27429  6280]
 [ 5021  8530]]
[[1056  905]
 [ 319 3922]]"""

'[[13501  2558]\n [ 2314  4312]]\n[[6340 1775]\n [1706 1125]]\n[[26734  6118]\n [ 4905  8163]]\n[[12280  1750]\n [ 1783  3713]]\n[[6597  789]\n [ 996 2172]]\n[[27429  6280]\n [ 5021  8530]]\n[[1056  905]\n [ 319 3922]]'